In [4]:
from BRScraper import nba
import pandas as pd 
import numpy as np

In [5]:
def select_data(desired_seasons, stats, salary, cap):
    df_f = pd.DataFrame()

    for desired_season in desired_seasons:
        # Selecting only salaries of S and S - 1
        salary1 = salary[['Player', desired_season,
                          str(int(desired_season[:4]) - 1) + '-' + str(int(desired_season[5:7]) - 1)]]
        
        stats1 = stats[stats["Season S-1"] == str(int(desired_season[:4]) - 1) + '-' + str(int(desired_season[5:7]) - 1)]

        df = stats1.merge(salary1, on = 'Player', how = 'left', validate = '1:1')

        df = df.rename(columns={str(int(desired_season[:4]) - 1) + '-' + str(int(desired_season[5:7]) - 1): 'Salary S-1',
                                  desired_season: 'Salary S'})
        
        # Removing null values
        df = df[(df['Salary S'] != -1) & (df['Salary S'].notna())]

        df['Year S'] = ('20' + ((df['Season S-1'].str[5: 7]).astype(int) + 1).astype(str)).astype(int)
        df['Year S-1'] = ('20' + (df['Season S-1'].str[5: 7]).astype(str)).astype(int)

        cap2 = cap.rename(columns = {'Cap Maximum': 'Cap Maximum S', 'Year': 'Year S'})
        df = df.merge(cap2, how = 'left', on = 'Year S', validate = 'm:1')

        cap2 = cap2.rename(columns = {'Cap Maximum S': 'Cap Maximum S-1', 'Year S': 'Year S-1'})
        df = df.merge(cap2, how = 'left', on = 'Year S-1', validate = 'm:1')

        df['% of Cap S-1'] = df['Salary S-1'] / df['Cap Maximum S-1'] * 100
        df['% of Cap S-1'][df['% of Cap S-1'] < 0] = -1

        df['% of Cap S'] = df['Salary S'] / df['Cap Maximum S'] * 100
        df['% of Cap S'][df['% of Cap S'] < 0] = -1

        # Get dummies for positions
        pos = pd.get_dummies(df['Pos']).astype(int)

        for col in pos.columns:
            if len(col) > 3:
                pos = pos.rename(columns = {col: col.split('-')[0]})

        pos = pos[['PG', 'SG', 'SF', 'PF', 'C']]
        pos = pos.groupby(pos.columns, axis = 1).sum()

        df = df.join(pos)
        df = df.drop(columns = {'Pos','Year S','Year S-1'}).reset_index(drop=True)

        df_f = pd.concat([df_f, df], ignore_index=True)
        
    df_f = df_f.fillna(0)
        
    return df_f

In [6]:
data_cap = {'Year': [2024],
            'Cap Maximum': [142.00]}
cap = pd.DataFrame(data_cap)

cap1 = pd.read_html('https://www.spotrac.com/nba/cba/')[0]
cap1 = cap1[['Year', 'Cap Maximum']]
cap1['Cap Maximum'] = (cap1['Cap Maximum'].str.replace('$', '').str.replace(',', '')).astype(float)/1000000

cap = pd.concat([cap, cap1], ignore_index = True)
cap = cap.astype({'Year': 'int32'})

cap.head()

,Year,Cap Maximum
0,2024,142.000
1,2023,136.021
2,2022,123.655
3,2021,112.414
4,2020,109.140


In [2]:
%store -r desired_seasons
%store -r stats_df
%store -r salaries

In [9]:
final_df = select_data(desired_seasons, stats_df, salaries, cap)
final_df.to_csv('final_data.csv', sep = ',', decimal = '.', index = False)
final_df.head()

C:\Users\nsalo\AppData\Local\Temp\ipykernel_17916\1560217967.py:29: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['% of Cap S-1'][df['% of Cap S-1'] < 0] = -1
C:\Users\nsalo\AppData\Local\Temp\ipykernel_17916\1560217967.py:29: SettingWith

,Player,Age,Tm,G,GS,MP_per_game,FG_per_game,FGA_per_game,FG%_per_game,3P_per_game,...,Salary S-1,Cap Maximum S,Cap Maximum S-1,% of Cap S-1,% of Cap S,C,PF,PG,SF,SG
0,Precious Achiuwa,23,TOR,55,12,20.7,3.6,7.3,0.485,0.5,...,2.840160,142.0,136.021,2.088031,3.084174,1,0,0,0,0
1,Steven Adams,29,MEM,42,42,27.0,3.7,6.3,0.597,0.0,...,17.926829,142.0,136.021,13.179457,8.873239,1,0,0,0,0
2,Bam Adebayo,25,MIA,75,75,34.6,8.0,14.9,0.540,0.0,...,30.351780,142.0,136.021,22.314040,22.957789,1,0,0,0,0
3,Ochai Agbaji,22,UTA,59,22,20.5,2.8,6.5,0.427,1.4,...,3.918360,142.0,136.021,2.880702,2.897324,0,0,0,0,1
4,Santi Aldama,22,MEM,77,20,21.8,3.2,6.8,0.470,1.2,...,-1.000000,142.0,136.021,-1.000000,1.545211,0,1,0,0,0
